In [0]:
%pip install mlflow==2.18.0 openai==1.61.0 databricks-agents httpx==0.27.2
%restart_python

Doing this with MLFLow=2.20.1 gives the following error `{"error_code": "BAD_REQUEST", "message": "Expected list, but got ndarray"}`. I had to pin at MLFlow = 2.18.0 (2.16 doesn't work nor does 2.19).

In [0]:
from mlflow.models import ModelConfig
config = ModelConfig(development_config='config.yaml')

import os
os.environ["API_SECRET_KEY"] = dbutils.secrets.get('shm', config.get("api_secret_key"))
os.environ["SEARCH_SECRET_KEY"] = dbutils.secrets.get('shm', config.get("search_secret_key"))

Load the Test Approaches and Questions

In [0]:
from pyfunc_azure_completion_retriever import AzureCompletionRetriever

approach = {
    'name': 'azurecompletionretriever',
    'file': 'pyfunc_azure_completion_retriever.py',
    'model': AzureCompletionRetriever(),
    'input_example': ['What is Databricks?'],
}

test_questions = [
  'What is Databricks?',
  'How does Databricks handle data storage?',
  'What are the key features of Databricks?',
  'How can I integrate Databricks with other tools?',
  'What is the Databricks Lakehouse Platform?',
  'How does Databricks support machine learning?',
  'What are the benefits of using Databricks?',
  'How can I secure my data in Databricks?',
  'What is the pricing model for Databricks?',
  'How can I get started with Databricks?',
  'What is the role of Apache Spark in Databricks?'
]

Test the approach and MLFlow Signature

In [0]:
import mlflow
from mlflow.models import infer_signature
from mlflow.models.signature import ModelSignature

model = approach['model']
input_example = approach['input_example']
model.predict(None, input_example)

signature = infer_signature(
  model_input=input_example,
  model_output=model.predict(None, input_example)
  )
signature

Time Raw Endpoint Calls via Class Only

In [0]:
import time
import numpy as np
import mlflow.pyfunc

# Run test questions against the reloaded model and track the time for each response
response_times = []
for question in test_questions:
    start_time = time.time()
    model.predict(None, [question])
    end_time = time.time()
    response_times.append(end_time - start_time)

print(f"Response Time {np.mean(response_times):.2f} +- {np.std(response_times):.2f}")

Can we predict on the whole list of questions? No. While we get a list in and out and the signature expects a list, the whole list goes in and we just take the last message.

In [0]:
model.predict(None, test_questions)

Log and Register the Model

In [0]:
with mlflow.start_run():
    # Set the registry URI to Unity Catalog if needed
    mlflow.set_registry_uri('databricks-uc')

    # Log the model in MLflow with the signature  
    logged_agent_info = mlflow.pyfunc.log_model(
        python_model=approach['file'],
        model_config='config.yaml',
        artifact_path="model",
        input_example=input_example,
        pip_requirements=[
            "mlflow==2.18.0",
            "openai==1.61.0", 
            "httpx==0.27.2",
            "databricks-sdk[openai]",
            "databricks-agents==0.12.0"
            ],
        signature=signature
        )

    print(f"Model logged with URI: {logged_agent_info.model_uri}")

Register the Model in UC and Deploy. Here we use the standard mlflow.deployments model because we don't have a compatible agents framework signature.

In [0]:
import mlflow.deployments
import requests

mlflow.set_registry_uri("databricks-uc")

model_name = f'shm.default.{approach["name"]}'

uc_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, 
    name=model_name
)

Reload the packaged model and profile local inference without serving

In [0]:
import mlflow.pyfunc

# Load the model
reloaded_model = mlflow.pyfunc.load_model(
    f"models:/{uc_model_info.name}/{uc_model_info.version}"
    )

reloaded_model.predict([question])

In [0]:
import time
import numpy as np

# Run test questions against the reloaded model and track the time for each response
response_times = []
for question in test_questions:
    start_time = time.time()
    response = reloaded_model.predict([question])
    end_time = time.time()
    response_times.append(end_time - start_time)

print(f"Response Time {np.mean(response_times):.2f} +- {np.std(response_times):.2f}")

Deploy the model after testing

In [0]:
api_secret_key_name = config.get("api_secret_key")
search_secret_key_name = config.get("search_secret_key")

client = mlflow.deployments.get_deploy_client("databricks")
serving_endpoint_name = f'shm-default-{approach["name"]}'

endpoint_config = {
    "served_entities": [{
        "entity_name": uc_model_info.name,
        "entity_version": uc_model_info.version,
        "workload_size": "Small",
        "scale_to_zero_enabled": "true",
        "environment_vars": {
            "API_SECRET_KEY": f"{{{{secrets/shm/{api_secret_key_name}}}}}",
            "SEARCH_SECRET_KEY": f"{{{{secrets/shm/{search_secret_key_name}}}}}"
        }
    }]
}

try:
    endpoint_info = client.get_endpoint(serving_endpoint_name)
    print(f"Endpoint {serving_endpoint_name} already exists, updating")
    client.update_endpoint(serving_endpoint_name, config=endpoint_config)
except requests.exceptions.HTTPError as e:
    try:
        client.delete_endpoint(serving_endpoint_name)
    except Exception as e:
        pass
    endpoint = client.create_endpoint(
            name=serving_endpoint_name,
            config=endpoint_config
            )

Test the Serving Endpoint Example

In [0]:
import time
import requests
import numpy as np
import mlflow.pyfunc

API_URL = f"https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/{serving_endpoint_name}/invocations"
API_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Run test questions against the reloaded model and track the time for each response
response_times = []
for question in test_questions:
    start_time = time.time()
    data = {"inputs": [question]}
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_TOKEN}"}
    response = requests.post(url=API_URL, json=data, headers=headers)
    end_time = time.time()
    response_times.append(end_time - start_time)

print(f"Response Time {np.mean(response_times):.2f} +- {np.std(response_times):.2f}")